<h1>Equal Weight S&P 500 Index Fund</h1>

The goal of this project is to create a Python script that accepts a portfolio (xlsx format) and returns the shares that we should purchase in order to create an equal weight version of the S&P 500 Index Fund

<h3>Library Imports</h3>

In [14]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [54]:
import warnings
warnings.filterwarnings('ignore')

<h3>Importing list of stocks</h3>

In [16]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


<h3>Creating the Pandas DataFrame to Store Stocks Data</h3>

The dataframe will contain the following columns:
<ul>
    <li>Price</li>
    <li>Market Capitalization</li>
    <li>Number of shares to buy</li>
</ul>

In [62]:
columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
df = pd.DataFrame(columns=columns)

<h3>Accessing the IEX Cloud API</h3>

We will use the following information from the API:
<ul>
    <li>Market capitalization for each stock</li>
    <li>Price of each stock</li>
</ul>

The IEX Cloud API allows a batch request of 100 Tickers at a time. We need to split our list into multiple lists of 100 Tickers (or less). Here is a helper function that does this:

In [24]:
'''
Function that splits a list (l) into multiple lists of (n) items
args:
    l: array
    n: number
returns 2D array
'''
def split_list(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [29]:
# Split our symbol list into multiple list of 100 symbols
symbols = stocks["Ticker"]
symbol_lists = list(split_list(symbols, 100))

In order to execute a batch request with the IEX Cloud API, we need to enter the stocks in the URL separated by a comma.<br>

<b>Example</b>: [AAL, AAPL, ABBV] becomes "AAL,AAPL,AABBV"

In [38]:
symbol_batches = [] # the array that contains the symbols separated by commas
for symbol_list in symbol_lists:
    symbol_batches.append(",".join(symbol_list))

Get the key to access the API

In [39]:
from secret_keys import IEX_CLOUD_API_TOKEN

In [63]:
# Loop over the stock symbols
for symbol_batch in symbol_batches:
    # Retrieve the data for each batch of symbols we're looping through
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_batch}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    # Loop over the each symbol in the batch we're currently in
    for symbol in symbol_batch.split(","):
        try:
            symbol_data = data[symbol]["quote"]
            # Add the data of the current symbol to our Data Frame
            df = df.append(pd.Series([
                symbol,
                symbol_data["latestPrice"],
                symbol_data["marketCap"],
                "N/A"
            ], index=columns), ignore_index=True)
        except:
            pass

In [65]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,125.09,37880563239,N/A
1,AAL,14.40,9199077660,N/A
2,AAP,189.40,11269626694,N/A
3,AAPL,147.33,2319272037574,N/A
4,ABBV,153.39,271696790975,N/A
...,...,...,...,...
496,YUM,114.53,32673472845,N/A
497,ZBH,111.88,23678201517,N/A
498,ZBRA,319.68,16552372106,N/A
499,ZION,53.73,8441958360,N/A


<h3>Calculate the number of shares to buy</h3>

Get the user's portfolio size by prompting an input box

In [72]:
def get_portfolio_size():
    portfolio_size = input("Enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print("\nYou need to enter a number.")
        val = get_portfolio_size()
    
    return val

In [75]:
portfolio_size = get_portfolio_size()

Enter the value of your portfolio: 1000000


Get the <b>position size</b>: the size we allocate to each stock

In [77]:
position_size = portfolio_size / len(df.index)
position_size

1996.007984031936

Populate the dataframe with the number of stocks we can buy for each Ticker

In [79]:
for i in range(0, len(df.index)):
    # Note: we can only buy full shares, so we use math.floor to get an integer without going over budget
    df.loc[i, columns[3]] = math.floor(position_size / df.loc[i, "Stock Price"])
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,125.09,37880563239,15
1,AAL,14.40,9199077660,138
2,AAP,189.40,11269626694,10
3,AAPL,147.33,2319272037574,13
4,ABBV,153.39,271696790975,13
...,...,...,...,...
496,YUM,114.53,32673472845,17
497,ZBH,111.88,23678201517,17
498,ZBRA,319.68,16552372106,6
499,ZION,53.73,8441958360,37


<h3>Export the Data Frame to an Excel File Traders can use</h3>

Initialize the XlsxWriter Object

In [98]:
writer = pd.ExcelWriter("equal_weight_s&p_500_recommendations.xlsx", engine="xlsxwriter")

Pass the Data Frame to the Writer Object

In [99]:
SHEET_NAME = "Recommended Trades"
df.to_excel(writer, SHEET_NAME, index=False)

Format the excel file with the following rules:
<ul>
    <li><b>Tickers:</b> String format</li>
    <li><b>Stock Prices:</b> \$XX.XXX</li>
    <li><b>Market Capitalization:</b> \$XX,XXX</li>
    <li><b>Number of Shares to Buy:</b> Integer</li>
</ul>

In [100]:
# Setup the styles
BACKGROUND_COLOR = "#0A0A23"
FONT_COLOR = "#FFFFFF"
BORDER_WIDTH = 1

STRING_FORMAT = writer.book.add_format({
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

PRICE_FORMAT = writer.book.add_format({
    "num_format": "$0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

VALUE_FORMAT = writer.book.add_format({
    "num_format": "$#,##0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

NUMBER_FORMAT = writer.book.add_format({
    "num_format": "0",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

map_columns_to_format = {
    "A": [columns[0], STRING_FORMAT],
    "B": [columns[1], PRICE_FORMAT],
    "C": [columns[2], VALUE_FORMAT],
    "D": [columns[3], NUMBER_FORMAT]
}

In [101]:
# Format Cells
for col in map_columns_to_format:
    title, cell_format = map_columns_to_format[col]
    
    # Format Header
    writer.sheets[SHEET_NAME].write(f"{col}1", title, STRING_FORMAT)
    
    # Format Body
    writer.sheets[SHEET_NAME].set_column(f"{col}:{col}", 18, cell_format)
writer.save()

Save the writer to the Excel File

In [102]:
writer.save()